## Cleaning Data

### Dataframes

In [3]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/s3_bucket_test/topics/0a9b5b8a2ae5.{}/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location.format("pin"))
df_geo = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location.format("geo"))
df_user = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location.format("user"))

display(df_pin)
# display(df_geo)
# display(df_user)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
home-decor,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",1,136k,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,6717,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"❤️ ❤️ MERRY CHRISTMAS ❤️ ❤️ ❤️ ❤️ Early Christmas Special:Buy 3 Get 1 Free, Buy 5 Get 2 Free,Deadline November 25. Color:GreenMaterial:Polyvinyl ChlorideItem Dimensions:LxWxH 20…",1,784,https://i.pinimg.com/originals/ef/40/7e/ef407e9568aa46fed4162bd1fd28786e.jpg,1676,image,paupoo,Local save in /data/christmas,"Christmas Hanging Baskets,Christmas Plants,Christmas Wreaths,Christmas Ornaments,Merry Christmas,Christmas Sale,Christmas Porch Ideas,Hanging Christmas Lights,Christmas Island",PAUPOO™ Pre-lit Artificial Christmas Hanging Basket - Flocked with Mixed Decorations and White LED Lights - Frosted Berry BUY 5 GET 2 FREE(7PACKS),3ed92c2d-9cca-4ccf-ac25-44a9d8bec919
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
vehicles,"By David Crane ; defrev (at) gmail (dot) com All photos contained in this article were shot by DefenseReview.com (DR), and are copyrighted. DefenseReview.com owns the copyright…",1,709,https://i.pinimg.com/originals/36/63/12/366312d747da1358397610a86bf21b20.jpg,10538,image,Ricky Lee,Local save in /data/vehicles,"Army Vehicles,Armored Vehicles,Cool Trucks,Cool Cars,Amphibious Vehicle,Offroader,Bug Out Vehicle,Vehicle Wraps,Terrain Vehicle",BC Customs (BCC) Search and Rescue Tactical Vehicle-5 (SRTV-5) Baja Racing-Type All-Terrain Combat Vehicle Armed/Weaponized with 7.62mm NATO Garwood Industries (GI) M134G Minigun/Gatling Gun: SXOR…,5d9fa7e2-2118-4442-99b6-537d60463a6a
event-planning,This fabulous DIY project made me drool when I first saw it and I knew immediately that I was going to have to make this! I absolutely love things like this...shiny sparkly thin…,1,985k,https://i.pinimg.com/originals/a6/79/3c/a6793c2e3deebca67ecd82b0087fc13c.jpg,4585,image,"DIY Joy - Crafts, Home Improvement, Decor & Recipes",Local save in /data/event-planning,"Cheap Favors,Wedding Favors Cheap,Wedding Invitations,Wedding Planning On 

### Cleaning df_pin dataframe

In [5]:
# remove duplicates 
df_pin = df_pin.drop_duplicates()

# replacing empty entries and entries with no relevant data with None
df_pin = df_pin.replace({"No description available Story format" : None}, subset=["description"])
df_pin = df_pin.replace({"User Info Error" : None}, subset=["follower_count"])
df_pin = df_pin.replace({"Image src error." : None}, subset=["image_src"])
df_pin = df_pin.replace({"User Info Error" : None}, subset=["poster_name"])
df_pin = df_pin.replace({"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e" : None}, subset=["tag_list"])
df_pin = df_pin.replace({"No Title Data Available" : None}, subset=["title"])

# transforming the follower_count column to make sure every entry is a number
df_pin = df_pin.withColumn("follower_count",regexp_replace("follower_count","k","000"))
df_pin = df_pin.withColumn("follower_count",regexp_replace("follower_count","M","000000"))

# casting follower_count column to int
df_pin = df_pin.withColumn("follower_count", df_pin["follower_count"].cast("int"))

# change the save_location column to only have the path
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# rename the index column
df_pin = df_pin.withColumnRenamed("index","ind")

# reordering the columns
df_pin = df_pin.select("ind","unique_id","title","description","follower_count","poster_name","tag_list","is_image_or_video","image_src","save_location","category")

display(df_pin)


ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
9093,8e478adb-0e3f-4045-83ce-79f155a63722,51 Unique Minimalist Tattoos Designs For Women,"""Minimalist Tattoos"" Discover how less is more with the best minimalist tattoo designs for women. Explore cool conservative simplicity with super simple ink ideas. See more idea…",54000,"Beautyholo | Latest Hairstyles, Nail Design Ideas, Home Décor DIY, Women Fashion Ideas","Small Tattoos Men,Tatoos Men,Cool Tattoos For Guys,Cool Tatoos For Women,Cool Simple Tattoos,Unique Tattoos,Diy Tattoo,Tattoo Fonts,Tattoo Quotes",image,https://i.pinimg.com/originals/c2/82/7d/c2827d11633b11c11a3da17b024264b0.jpg,/data/tattoos,tattoos
7801,55f282f7-3038-450c-9b72-366072583f1c,"Art Print: Wilson's Be Stronger Than Your Excuses, 32x24in.","Size: 32x24in Be Stronger Than Your ExcusesWe have more Brett Wilson Posters. Choose from our catalog of over 500,000 posters! This art print displays sharp, vivid images with a…",72000,AllPosters,"Motivation Positive,Fitness Motivation Quotes,Motivational Workout Quotes,Quotes About Fitness,Motivational Quotes For Working Out,Health Fitness Quotes,Gym Fitness,Motivating Quotes,Motivational Quotes For Athletes",image,https://i.pinimg.com/originals/d3/49/5e/d3495e71e97c681086ef12fbb10e4eb7.jpg,/data/quotes,quotes
3351,282a8cf2-d4f3-4d21-837c-90c953b737cf,Easy Christmas Crafts for Kids - Fruit Loop Candy Cane,"If you're looking for easy Christmas crafts for kids, this is it! You only need 2 things to make them - Fruit Loop cereal and pipe cleaners! They're a great fine motor activity…",29000,Lindsey Blogs,"Kids Food Crafts,Preschool Christmas Crafts,Christmas Crafts For Kids To Make,Christmas Activities For Kids,Christmas Tree Crafts,Toddler Christmas,Toddler Crafts,Preschool Winter,Winter Activities",image,https://i.pinimg.com/originals/f4/40/aa/f440aae805984076e0886407677e0ec2.png,/data/diy-and-crafts,diy-and-crafts
3930,da81fbde-43f4-4cb8-afb0-89f3d0514167,Student Led Discussion Strategies for Whole Class Discussion,"Every English language arts teacher needs a variety of successful, student led discussion strategies that will provide opportunities for student learning.",6000,Lindsay Ann Learning | High School English Activities,"Instructional Coaching,Instructional Strategies,Teaching Strategies,Teaching Tips,Teaching Reading,Student Teaching,Cooperative Learning Strategies,Teaching Art,Cooperative Games",image,https://i.pinimg.com/originals/9f/26/18/9f2618e5fc1a04d74844549f49876051.jpg,/data/education,education
3184,02acd2ca-a1fd-4290-9431-085dd78035a2,40 DIY Crafts To Make & Sell For Money,"40 Easy & fun DIY crafts to make and sell that you need to try!! If you are looking at making some extra cash, then make sure to try out these DIY crafts!! #diycrafts #selldiycr…",43000,"RT Lifestyle Magazine | Travel, DIY, Recipes, Fitness & More","Diy Gifts To Sell,Diy Crafts How To Make,Fun Diy Crafts,Sell Diy,Crafts For Teens,Decor Crafts,Homemade Crafts,Recycled Crafts,Crafts To Make And Sell Easy",image,https://i.pinimg.com/originals/61/61/5f/61615f747dab735ef8128fd3e029bcd2.png,/data/diy-and-crafts,diy-and-crafts
2780,e8bd0e91-9560-42cd-86c4-b754870a0e89,Quick & Easy Kids Halloween Crafts & Activities - Over the Big Moon,These Kids Halloween Crafts and Activities are perfect for Halloween and for kids of all ages - some require craft suppies and others are printable!,79000,Pam from Over the Big Moon,"Fall Crafts For Kids,Toddler Crafts,Kids Crafts,Arts And Crafts,Halloween Crafts For Kindergarten,Halloween Crafts For Kids To Make,Halloween Crafts For Preschoolers,Preschool Halloween Crafts,Winter Craft",image,https://i.pinimg.com/originals/f4/28/50/f428509f9ef819a04f6d2f8224233c22.jpg,/data/diy-and-crafts,diy-and-crafts
3716,74f878f3-6f1b-4b41-8e71-67a68a0ae76a,Engaging Secondary Students with Crime Stories — Bespoke ELA: Essay Writing Tips + Lesson Plans,"Here, you will find four projects that involve 

### Cleaning df_geo dataframe

In [7]:
# remove duplicates
df_geo = df_geo.drop_duplicates()

# coordinates column
df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))

# dropping latitude and logitude columns
df_geo = df_geo.drop("latitude","longitude")

# converting timestamp from a string to a timestamp
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))

# reordering the columns
df_geo = df_geo.select("ind","country","coordinates","timestamp")

display(df_geo)


ind,country,coordinates,timestamp
480,Mozambique,"List(-30.7158, -167.461)",2019-02-08T03:34:58.000+0000
6305,Azerbaijan,"List(-89.3669, -170.886)",2021-05-09T19:54:08.000+0000
5151,Guatemala,"List(-47.3372, -151.397)",2020-07-15T01:42:38.000+0000
10509,Gibraltar,"List(-67.187, -24.2977)",2019-03-06T12:21:56.000+0000
9234,Azerbaijan,"List(-68.2139, -44.4492)",2021-07-08T05:58:46.000+0000
9936,Australia,"List(-82.1715, -147.691)",2018-06-30T22:07:13.000+0000
6809,Bangladesh,"List(-89.4254, -161.818)",2020-04-17T13:20:43.000+0000
10673,Nicaragua,"List(37.1991, -32.1713)",2021-06-30T08:47:01.000+0000
2315,Guinea-Bissau,"List(17.806, -1.9042)",2019-06-05T08:54:04.000+0000
5468,Isle of Man,"List(-83.544, 33.9473)",2020-05-16T23:17:50.000+0000


### Cleaning df_user dataframe

In [9]:
# removing duplicates
df_user = df_user.drop_duplicates()

# a new user_name column
df_user = df_user.withColumn("user_name", concat("first_name", lit(" "), "last_name"))

# dropping first_name and last_name
df_user = df_user.drop("first_name", "last_name")

# converting date_joined to a timestamp
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))

# reordering the columns
df_user = df_user.select("ind", "user_name", "age", "date_joined")

display(df_user)


ind,user_name,age,date_joined
9426,Nicholas Mcdaniel,45,2017-05-24T01:41:26.000+0000
9234,Christina Aguilar,23,2015-12-09T20:29:10.000+0000
8887,Austin Rodriguez,24,2016-03-31T20:56:39.000+0000
10837,Mark Richardson,55,2015-11-18T05:46:35.000+0000
7285,Alejandra Acevedo,20,2015-11-24T21:01:23.000+0000
9332,Gregory Carpenter,23,2016-07-14T11:45:49.000+0000
4765,Danielle Gonzalez,36,2017-02-16T16:01:34.000+0000
9099,Gregory Carpenter,23,2016-07-14T11:45:49.000+0000
10369,Gabrielle Flores,34,2017-09-17T01:53:38.000+0000
2810,Brandi Cervantes,35,2016-05-31T01:40:44.000+0000


---

## Data Query

In [12]:
# creating temporary views
df_pin.createOrReplaceTempView("pin")
df_geo.createOrReplaceTempView("geo")
df_user.createOrReplaceTempView("user")

#### Find the most popular category in each country

In [14]:
%sql

SELECT country, category, category_count
FROM(
    SELECT country, category, COUNT(category) AS category_count,
    ROW_NUMBER() OVER(PARTITION BY country ORDER BY COUNT(category) DESC) AS row_number
    FROM geo
    JOIN pin ON pin.ind = geo.ind
    GROUP BY country, category
)
WHERE row_number = 1;

country,category,category_count
Afghanistan,education,6
Albania,art,6
Algeria,quotes,12
American Samoa,tattoos,5
Andorra,tattoos,5
Angola,diy-and-crafts,3
Anguilla,diy-and-crafts,2
Antarctica (the territory South of 60 deg S),tattoos,2
Antigua and Barbuda,art,3
Argentina,tattoos,6


#### Find which was the most popular category each year

In [16]:
%sql

SELECT post_year, category, category_count FROM (
  SELECT year(timestamp) AS post_year, category, COUNT(category) AS category_count,
  ROW_NUMBER() OVER (PARTITION BY year(timestamp) ORDER BY COUNT(category) DESC) AS row_number
  FROM geo 
  JOIN pin on pin.ind = geo.ind
  GROUP BY year(timestamp), category
)
WHERE 2018 <= post_year AND post_year <= 2022 AND row_number = 1;


post_year,category,category_count
2018,education,13
2019,diy-and-crafts,12
2020,mens-fashion,13
2021,tattoos,12
2022,christmas,16


#### Find the user with most followers in each country

Step 1:

In [19]:
%sql

SELECT country, poster_name, max_follower_count FROM (
  SELECT country, poster_name, MAX(follower_count) AS max_follower_count,
  ROW_NUMBER() OVER (PARTITION BY country ORDER BY MAX(follower_count) DESC) as row_number
  FROM pin 
  JOIN geo ON geo.ind = pin.ind
  GROUP BY country,poster_name
)
WHERE row_number = 1;

country,poster_name,max_follower_count
Afghanistan,9GAG,3000000
Albania,The Minds Journal,5000000
Algeria,YourTango,942000
American Samoa,Mamas Uncut,8000000
Andorra,Teachers Pay Teachers,1000000
Angola,Tastemade,8000000
Anguilla,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",92000
Antarctica (the territory South of 60 deg S),Refinery29,1000000
Antigua and Barbuda,Country Living Magazine,1000000
Argentina,Next Luxury,800000


Step 2:

In [21]:
%sql

SELECT country, MAX(follower_count) AS max_follower_count
FROM pin
JOIN geo ON geo.ind = pin.ind
GROUP BY country
ORDER BY MAX(follower_count) DESC
LIMIT 1;

country,max_follower_count
Angola,8000000


#### Find the most popular category for different age groups

In [23]:
%sql

WITH age_table AS (
  SELECT age, pin.category AS category
  FROM user
  JOIN pin ON pin.ind = user.ind
), age_group_table AS (
  SELECT CASE 
    WHEN 18 <= age AND age <= 24 THEN '18-24'
    WHEN 25 <= age AND age <= 35 THEN '25-35'
    WHEN 36 <= age AND age <= 50 THEN '36-50'
    WHEN 50 < age THEN '50+'
  END AS age_group,
  category, COUNT(category) AS category_count
  FROM age_table
  GROUP BY age_group, category
)
SELECT age_group, category, category_count
FROM (
  SELECT  age_group, 
          category, 
          category_count,
          ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS row_number
  FROM 
    age_group_table
)
WHERE row_number = 1;

age_group,category,category_count
18-24,tattoos,31
25-35,travel,20
36-50,quotes,13
50+,mens-fashion,6


#### Find the median follower count for different age groups

In [25]:
%sql
SELECT CASE 
    WHEN 18 <= age AND age <= 24 THEN '18-24'
    WHEN 25 <= age AND age <= 35 THEN '25-35'
    WHEN 36 <= age AND age <= 50 THEN '36-50'
    WHEN 50 < age THEN '50+'
  END AS age_group, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
  FROM user 
  JOIN pin ON pin.ind = user.ind
  GROUP BY age_group;

age_group,median_follower_count
50+,954.0
36-50,6500.0
18-24,134000.0
25-35,26000.0


#### Find how many users have joined each year

In [27]:
%sql
SELECT year(date_joined) AS post_year, COUNT(date_joined) AS number_usesrs_joined
FROM user
GROUP BY post_year
HAVING post_year >= 2015 AND post_year <= 2020;

post_year,number_usesrs_joined
2015,210
2016,262
2017,92


#### Find the median follower count of users based on their joining year

In [29]:
%sql
SELECT year(date_joined) AS post_year, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
FROM user
JOIN pin ON pin.ind = user.ind
GROUP BY post_year
HAVING post_year >= 2015 AND post_year <= 2020;

post_year,median_follower_count
2015,131000.0
2016,19000.0
2017,3000.0


#### Find the median follower count of users based on their joining year and age group

In [31]:
%sql
SELECT CASE 
    WHEN 18 <= age AND age <= 24 THEN '18-24'
    WHEN 25 <= age AND age <= 35 THEN '25-35'
    WHEN 36 <= age AND age <= 50 THEN '36-50'
    WHEN 50 < age THEN '50+'
  END AS age_group, year(date_joined) AS post_year, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
  FROM user
  JOIN pin ON pin.ind = user.ind
  GROUP BY post_year, age_group
  HAVING post_year >= 2015 AND post_year <= 2020;


age_group,post_year,median_follower_count
25-35,2015,42000.0
18-24,2016,46000.0
50+,2015,14000.0
18-24,2015,301000.0
25-35,2016,21500.0
36-50,2017,3000.0
50+,2016,908.0
50+,2017,861.0
25-35,2017,1000.0
18-24,2017,13500.0
